In [63]:
import pyximport; pyximport.install()
from csvparser import *
from gpcode import *

In [64]:
csv_filename = "Comparison Results/mapping-" + machine_alias + ".csv.txt"

fields = [['Heuristic'], ['Complexity'], ['Fitness']]
conditions = []

csv_data = extractDictFromCSV(csv_filename)
parsed_data = extractIndividualsFromData(csv_data, ind_field=fields[0], complexity_field= fields[1], fitness_field=fields[2], condition=conditions)
parsed_data = sorted(parsed_data, reverse=True)

rank_dict = {}

for i in range(0, len(parsed_data)):
    rank_dict[str(parsed_data[i].heuristic)] = i

fitness_by_complexity_dict = {}
for d in parsed_data:
    c = int(d.complexity)
    if c not in fitness_by_complexity_dict:
        fitness_by_complexity_dict[c] = []
    fitness_by_complexity_dict[c].append(float(d.fitness))

map_complexity_to_key = dict()
keys = fitness_by_complexity_dict.keys()
keys = sorted(keys)

for i in range(0, len(keys)):
    map_complexity_to_key[keys[i]] = i

In [65]:
top_heuristic_by_complexity_candidates_list = []

for k in fitness_by_complexity_dict.keys():
    top_heuristic_by_complexity_candidates_list.append((k, max(fitness_by_complexity_dict[k])))

list_of_top_heuristics = []

for p in parsed_data:
    for h in top_heuristic_by_complexity_candidates_list:
        if int(p.complexity) == h[0] and float(p.fitness) == h[1]:
            list_of_top_heuristics.append(p.heuristic)

In [67]:
match_csv_filename = "MatchUp Results/mapping-" + machine_alias + "-2018-02-01 11:49:34.579895.csv.txt"

conditions = []

match_csv_data = extractDictFromCSV(match_csv_filename)
match_parsed_data = extractIndividualsFromMatchUpData(match_csv_data)

heuristic_set = set()
for match in match_parsed_data:
    heuristic_set.add(match.heuristic_p1)
    heuristic_set.add(match.heuristic_p2)

In [68]:
def approximateComplexity(heuristic_str):
    return findComplexity(gp.PrimitiveTree.from_string(heuristic_str, pset))

In [69]:
template = []
for i in range(0, len(list_of_top_heuristics)):
    template.append(None)
    
results = []
for i in range(len(list_of_top_heuristics)):
    results.append(list(template))
    
for m in match_parsed_data:
    if m.heuristic_p1 in list_of_top_heuristics and m.heuristic_p2 in list_of_top_heuristics:
        c_p1 = approximateComplexity(m.heuristic_p1)
        c_p2 = approximateComplexity(m.heuristic_p2)
        results[map_complexity_to_key[c_p1]][map_complexity_to_key[c_p2]] = m.fitness_p1
        results[map_complexity_to_key[c_p2]][map_complexity_to_key[c_p1]] = m.fitness_p2
        #results[rank_dict[m.heuristic_p1]][rank_dict[m.heuristic_p2]] = m.fitness_p1
        #results[rank_dict[m.heuristic_p2]][rank_dict[m.heuristic_p1]] = m.fitness_p2

In [62]:
temp_dict = dict()
for m in match_parsed_data:
    if m.heuristic_p1 not in temp_dict:
        temp_dict[m.heuristic_p1] = 0
    if m.heuristic_p2 not in temp_dict:
        temp_dict[m.heuristic_p2] = 0
    
    temp_dict[m.heuristic_p1] += m.fitness_p1
    temp_dict[m.heuristic_p2] += m.fitness_p2

eval_csv_filename = "Comparison Results/sum_money_eval-" + machine_alias + ".csv.txt"
createLogFile(eval_csv_filename, "Heuristic; Complexity; Fitness;\n")

for k in temp_dict:
    logToFile(eval_csv_filename, str(k) + "; " + str(approximateComplexity(k)) + "; " + str(temp_dict[k]) + ";\n")

In [71]:
##### TOP HEURISTIC OF EACH COMPLEXITY

import pandas as pd
import numpy as np

def highlight_max(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    if s > 0:
        return 'background-color: green'
    if s < 0:
        return 'background-color: red'
    return ''

temp = []
for x in results:
    temp.append([])
    for y in x:
        if y != None:
            temp[-1].append(-1 * y)
        else:
            temp[-1].append(0)
            
df = pd.DataFrame()
df = pd.concat([df, pd.DataFrame(temp, columns=keys, index=keys)],
               axis=1)
df = df.transpose()
#df.iloc[0, 2] = np.nan
df.style
s = df.style.applymap(highlight_max)

#s = df.style.applymap(color_negative_red)
s

In [58]:
for i in range(0, len(results)):
    print (keys[i], sum(results[i][0:i] + results[i][i+1:]))

print "+++++++++++++++++++++++"
print "+++++++++++++++++++++++"
    
for i in range(0, len(results)):
    temp = results[i][0:i] + results[i][i+1:]
    print (keys[i], sum([1 if x > 0.0 else 0 for x in temp]))

(3, -8.835425)
(4, -7.46355)
(5, -0.5745750000000001)
(6, 1.8648750000000995)
(7, 6.281425)
(8, -9.802600000000002)
(9, 2.9875249999999802)
(10, -1.42215)
(11, 3.69819999999995)
(13, 3.7097)
(14, 3.06797500000006)
(15, 3.7880499999997603)
(16, 2.70055000000015)
+++++++++++++++++++++++
+++++++++++++++++++++++
(3, 1)
(4, 2)
(5, 4)
(6, 5)
(7, 12)
(8, 0)
(9, 9)
(10, 3)
(11, 8)
(13, 10)
(14, 8)
(15, 10)
(16, 6)
